# Interactive charts

For this, we will be using IPython widgets to make plots interactive. As usual, import the libraries we need:

In [1]:
# Import Widgets
import ipywidgets as widgets

# Import the interact and the interact_manual from widgets
from ipywidgets.widgets import interact, interact_manual

# Import Pandas
import pandas as pd

# Import Plotly
import plotly.graph_objects as go

As the first example of a widget, let's begin with creating a **Slider**:

In [2]:
slider = widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='Slider:',
    value=3
)

slider

IntSlider(value=3, description='Slider:', max=10)

In the code above, we created an integer **Slider** with values from zero to ten. The initial value is set to three. By running the code we can see an interactive slider, but we still don't have its value. We can get this value by accessing the values attribute of our variable:

In [4]:
# The current value of the slider
print(slider.value)

3


There are many interactive widgets we can use in Jupyter notebook. 
You can view a full list of widgets [here](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html)

In [5]:
# Show the widgets directory
print(dir(widgets))

['Accordion', 'AppLayout', 'Audio', 'BoundedFloatText', 'BoundedIntText', 'Box', 'Button', 'ButtonStyle', 'CallbackDispatcher', 'Checkbox', 'Color', 'ColorPicker', 'Combobox', 'Controller', 'CoreWidget', 'DOMWidget', 'DatePicker', 'Datetime', 'Dropdown', 'FileUpload', 'FloatLogSlider', 'FloatProgress', 'FloatRangeSlider', 'FloatSlider', 'FloatText', 'GridBox', 'GridspecLayout', 'HBox', 'HTML', 'HTMLMath', 'Image', 'IntProgress', 'IntRangeSlider', 'IntSlider', 'IntText', 'Label', 'Layout', 'NumberFormat', 'Output', 'Password', 'Play', 'RadioButtons', 'Select', 'SelectMultiple', 'SelectionRangeSlider', 'SelectionSlider', 'SliderStyle', 'Style', 'Tab', 'Text', 'Textarea', 'ToggleButton', 'ToggleButtons', 'ToggleButtonsStyle', 'TwoByTwoLayout', 'VBox', 'Valid', 'ValueWidget', 'Video', 'Widget', '__builtins__', '__cached__', '__doc__', '__file__', '__jupyter_widgets_base_version__', '__jupyter_widgets_controls_version__', '__loader__', '__name__', '__package__', '__path__', '__protocol_vers

### Filter a DataFrame with Widgets
We will explore how to use widgets to control a dataframe. The sample dataset we have chosen is the ```'Number of International Visitors to London'```, which shows the totals of London's visitors with regards to nights spent, visits, and spend **broken down by year, quarter, purpose, duration, mode, and country.**
We will use the first 200 rows

In [6]:
# the data url
url = "https://data.london.gov.uk/download/number-international-visitors-london/b1e0f953-4c8a-4b45-95f5-e0d143d5641e/international-visitors-london-raw.csv"

# create the data-frame
df_london = pd.read_csv(url, encoding= 'unicode_escape')
df_london.columns = ["year","quarter","market","dur_stay","mode","purpose","area","visits","spend","nights","sample"]

# sample 200 rows
df = df_london.sample(200)

Check out the DF

In [7]:
df.head()

,year,quarter,market,dur_stay,mode,purpose,area,visits,spend,nights,sample
7584,2004,April-June,Netherlands,8-14 nights,Air,Holiday,LONDON,1.249270,0.616047,13.256700,1
46067,2015,April-June,Malaysia,8-14 nights,Air,VFR,LONDON,5.060668,2.208552,41.357692,6
7485,2004,April-June,France,4-7 nights,Air,VFR,LONDON,5.105014,1.997904,24.167620,7
14925,2006,April-June,Mexico,4-7 nights,Air,Study,LONDON,0.456575,0.140625,3.196020,1
53488,2017,July-September,Switzerland,15+ nights,Air,VFR,LONDON,0.883522,0.812213,16.884468,1


Now, imagine we want to filter this based on selected columns and the corresponding threshold. We can change the filter condition in the code every time we want to filter the data-frame or create an interactive widget. Let's go with the second option and define a function that filters the data-frame based on the selected column and threshold:

In [8]:
# The filter function
def filter_df(column, threshold):
    return df[df[column] <= threshold]

The only thing left is to create a widget to set the parameters of the function. Let's say we want to use the columns 'spend' and 'visits' in our dropdown widget and values from the interval \<0,30> with step 1 in a slider widget:

In [9]:
filter_widget = widgets.interact(filter_df,
                                 column=['spend','visits'],
                                 threshold=(1, 30, 1))

interactive(children=(Dropdown(description='column', options=('spend', 'visits'), value='spend'), IntSlider(va…

We created a filter_widget variable that stores our widget. We can use this widget in future code by the following command:

In [10]:
# Use filter later in code
filter_widget.widget

interactive(children=(Dropdown(description='column', options=('spend', 'visits'), value='spend'), IntSlider(va…

### Interactive Plots with Widgets
In this section, we will learn how to change the x- and y-axis data with widgets. Let's define a function that creates a scatter plot from the selected columns of a data-frame:

In [12]:
@interact
def scatter_plot(x=list(df.select_dtypes('number').columns),
                 y=list(df.select_dtypes('number').columns)[1:]):
    # trace
    trace = [go.Scatter(x=df[x],y=df[y],mode='markers')]
    
    # layout
    layout = go.Layout(
        title = 'Scatter plot', # Graph Title
        xaxis = dict(title = x.title()), # x-axis label
        yaxis = dict(title = y.title()), # y-axis label
        hovermode = 'closest'  # handles multiple points landing on same vertical
    )
    
    #fig
    fig = go.Figure(trace, layout)
    fig.show()

interactive(children=(Dropdown(description='x', options=('visits', 'spend', 'nights', 'sample'), value='visits…

When we change the column choices in the drop down widget, our plot changes almost immediately. Dropdowns are generated by the ```@interact decorator``` of our scatter_plot function. However, this fast reaction is caused by using only 200 rows of data from the original set. For bigger frames, this real time change could be a problem. The solution is the ```@interact_manual``` decorator that creates dropdowns in the same way, but also creates a button that allows us to confirm the selection. Let's try to use this decorator:

In [13]:
@interact_manual
def scatter_plot(x=list(df.select_dtypes('number').columns), 
                 y=list(df.select_dtypes('number').columns)[1:]):

    # trace
    trace = [go.Scatter(x=df[x], y=df[y], mode='markers')]

    # layout
    layout = go.Layout(
                title = 'Scatter plot', # Graph title
                xaxis = dict(title = x.title()), # x-axis label
                yaxis = dict(title = y.title()), # y-axis label
                hovermode ='closest' # handles multiple points landing on the same vertical
    )

    # fig
    fig = go.Figure(trace, layout)
    fig.show()

interactive(children=(Dropdown(description='x', options=('visits', 'spend', 'nights', 'sample'), value='visits…

This gives both the computer a break and saves time, as the chart only updates when the run button is pushed.